In [ ]:
!pip install diffusers transformers accelerate torch

In [ ]:
import torch
from diffusers import AutoencoderKL, UNet2DConditionModel, DDIMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
from PIL import Image
import requests
#모델 stable-diffusion
model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda" # GPU 사용 설정


vae = AutoencoderKL.from_pretrained(model_id, subfolder="vae").to(device) #잠재 공간 변환 모델
tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder="tokenizer") #텍스트 토큰화 모델
text_encoder = CLIPTextModel.from_pretrained(model_id, subfolder="text_encoder").to(device)#텍스트 인코더
unet = UNet2DConditionModel.from_pretrained(model_id, subfolder="unet").to(device)#unet모델
scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")

In [ ]:
# 1. Denoising을 테스트할 이미지 불러오기
# 예시: URL에서 이미지 로드
#url = "https://thumbnail10.coupangcdn.com/thumbnails/remote/492x492ex/image/retail/images/2024/04/12/11/2/1cf43ccc-e294-4280-a431-f4ca4ed571c7.jpg"
url = "https://vrthumb.clipartkorea.co.kr/2023/10/12/tc00240107234.jpg"
response = requests.get(url, stream=True)
init_image = Image.open(response.raw).convert("RGB").resize((512, 512))

# 2. 사용할 텍스트 프롬프트 정의
#prompt = "A cartoon character is Crayon shin chan who is five years old, wears a orange full dress,and black headphone,he is listening the music"
prompt="a white cat jumping on the grass"

init_image


In [ ]:
from torchvision import transforms

# 1. 이미지 전처리 -> VAE에 입력 가능한 텐서 형태로 변환
preprocess = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ]
)
image_tensor = preprocess(init_image).unsqueeze(0).to(device)


# 2. 프롬프트 전처리 -> 텍스트 인코더에 입력 가능한 형태로 변환
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
with torch.no_grad():
    text_embeddings = text_encoder(text_input.input_ids.to(device))[0]

In [ ]:
# 1. VAE로 z₀ 생성 (원본 이미지의 잠재 벡터)
with torch.no_grad():
    # .sample()을 사용하여 분포의 평균(mean)을 샘플링
    init_latents = vae.encode(image_tensor).latent_dist.sample()
    init_latents = 0.18215 * init_latents # 매직 넘버 (모델 학습 시 사용된 스케일 값)


# 2. 노이즈 추가
noise = torch.randn_like(init_latents) # 잠재 벡터와 동일한 크기의 랜덤 노이즈 생성
timestep = torch.tensor([800]).to(device) # 시작 타임스텝 (가장 노이즈가 심한 상태)
noisy_latents = scheduler.add_noise(init_latents, noise, timestep)

In [ ]:
# Denoising 설정
scheduler.set_timesteps(800) # 총 1000번의 스텝 설정
latents = noisy_latents.clone() # noisy_latents 복사해서 사용

intermediate_results = {} # 중간 결과를 저장할 딕셔너리
# Denoising Loop
for t in scheduler.timesteps:

    #999,998,997....0
    # UNet에 입력: 노이즈 낀 잠재 벡터, 타임스텝, 텍스트 임베딩
    # The UNet call is fine, it expects a tensor.
    with torch.no_grad():
        noise_pred = unet(latents, t, encoder_hidden_states=text_embeddings).sample

    # scheduler.step으로 노이즈 제거 및 이전 단계 잠재 벡터 계산
    # Pass 't' as an integer using .item()
    latents = scheduler.step(noise_pred, t.item(), latents).prev_sample # 👈 .item() added here

    # 지정된 타임스텝에서 중간 결과 저장
    if t.item() in [600,450, 350,200, 1]:
        intermediate_results[t.item()] = latents.clone()

In [ ]:
import numpy as np

# VAE 디코더로 잠재 벡터를 이미지로 변환하는 함수
def decode_latents(latents):
    latents = 1 / 0.18215 * latents
    with torch.no_grad():
        image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    return pil_images

# 저장된 중간 결과들을 이미지로 변환하고 출력
for t, latent in sorted(intermediate_results.items(), reverse=True):
    print(f"--- Timestep {t} ---")
    image = decode_latents(latent)[0]
    display(image)